In [0]:
import torch
!pip3 install numpy scipy librosa unidecode inflect librosa
!pip3 install librosa --upgrade
import librosa
import librosa.core
tacotron2 = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_tacotron2')

In [0]:
tacotron2 = tacotron2.to('cuda').eval()
!pip3 install librosa --upgrade


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
def text_to_spectrogram(text):
    sequence = np.array(tacotron2.text_to_sequence(text, ['english_cleaners']))[None, :]
    sequence = torch.from_numpy(sequence).to(device='cuda', dtype=torch.int64)
    _, mel, _, _ = tacotron2.infer(sequence)
    return mel.detach().cpu().numpy()[0]

In [0]:
def griffin_lim_algorithm(S, iterations, hop_length=256):
    # at each step, 
    # compute S_ = stft(x)
    # take angle from S_ - _P = np.angle(S_)
    # replace magnitude of S_ with S - D = S * np.exp(1j * _P)
    # calculate x = istft(S)

    phase = np.random.randn(*S.shape)
    x = librosa.istft(S * np.exp(1j * phase),hop_length=hop_length, win_length=hop_length*4)

    for k in range(iterations):
        computed_S = librosa.stft(x, n_fft=1024, hop_length=hop_length, win_length=hop_length*4)
        phase = np.angle(computed_S)

        augmented_S = S * np.exp(1j * phase)
        x = librosa.istft(augmented_S, hop_length=hop_length, win_length=hop_length*4)

    return x
def mel_spectrogram_to_audio(S):
    S_ = librosa.core.db_to_power(S)
    S_ = librosa.feature.inverse.mel_to_stft(S_, n_fft=1024, power=0.2)
    librosa.display.specshow(S_)

    y = griffin_lim_algorithm(S_, 50)
    return y
spectrogram1 = text_to_spectrogram("My name is Alex and I like signals.")
audio = mel_spectrogram_to_audio(spectrogram1)
from IPython.display import Audio
Audio(audio, rate=22050)